In [1]:
def big_data(X,y,reqlen):
    Big_X = np.array(X)
    Big_Y = np.array(y)
    while len(Big_X) < reqlen:
        Big_X = np.concatenate((Big_X, Big_X))
        Big_Y = np.concatenate((Big_Y, Big_Y))
    print(Big_X.shape)
    indices = np.random.choice(Big_X.shape[0], reqlen, replace=False)
    new_X = Big_X[indices]
    new_Y = Big_Y[indices]
    return new_X, new_Y

# 1000 samples

In [2]:
from sklearn.datasets import load_boston
import sklearn.ensemble
import numpy as np
from sklearn.model_selection import train_test_split
from mosaicml import *
from mosaicml.constants import MLModelFlavours

import pandas as pd
from pandas import read_csv, DataFrame
from sklearn.preprocessing import Imputer
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from mosaicml.api import graph

# Importing the dataset
boston = load_boston()

# creating the model object
rf = sklearn.ensemble.RandomForestRegressor(n_estimators=10)

X = boston.data
Y = boston.target

Big_X, Big_Y = big_data(X, Y,1000)

print(Big_X.shape)
print(Big_Y.shape)

train, test, labels_train, labels_test = train_test_split(Big_X, Big_Y, test_size=0.2, random_state=0)

rf.fit(train, labels_train)


# @scoring_func
def score(model, request):
    payload = request.json["payload"]
    data_list = payload
    data_array = np.asarray(data_list)
    try:
        prediction = model.predict(data_array)
    except:
        prediction = model.predict(data_array.reshape(1, -1))
    return prediction.tolist()

# to verify if score func works
import requests
req = requests.Request()
req.json = {"payload":train[10].tolist()}

score(rf,req)

from datetime import datetime
st = datetime.utcnow()
print(st,"\n")

y_pred=rf.predict(test)

register_model(rf,
                score,
                "boston_regression_1k",
                "boston_regression_1k",
                MLModelFlavours.sklearn,
                init_script="mkdir abc \\n cd abc \\n touch abc.txt",
                input_type="json",
                explain_ai=True,
               x_train=train,
               y_train=labels_train, 
               y_pred=y_pred,
               y_true=labels_test,
               features=list(boston.feature_names),
               original_features=list(boston.feature_names),
               model_type="regression",
               feature_names=list(boston.feature_names), 
               feature_ids=list(boston.feature_names), 
               target_names=['Housing_Price']
              )

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: http://mosaic-ai-backend:5000/registry/api/v1/ml-model/metrics

In [ ]:

en = datetime.utcnow()
print(en,"\n")
diff = en - st
print(str(diff))

